# Using the generated SDK and data models

## Preparation

The main entry point for the SDK is the `PowerOpsClient`

To get started, we assume you have installed the `cognite-power-ops` SDK and that all configurations have been setup in Cognite Data Fusion (CDF).

Furthermore, it is assumed that you have setup one or two `toml` files with the credentials for connecting to Cognite Data Fusion and settings for this SDK.


### Interactive Login
This only requires one `toml` file looking like this

`settings.toml`

```toml
[cognite]
  login_flow = "interactive"
  project = "<cdf-project>"
  tenant_id = "<tenant-id>"
  cdf_cluster = "<cdf-cluster>"
  client_id = "<client-id>"

[powerops]
  read_dataset = "uc:000:powerops"
  write_dataset = "uc:000:powerops"
  monitor_dataset = "uc:po:monitoring"

```

### Client Credentials Login

For this case the you can use two toml files to separate the secrets from the regular settings.


`settings.toml`

```toml
[cognite]
  login_flow = "client_credentials"
  project = "<cdf-project>"
  tenant_id = "<tenant-id>"
  cdf_cluster = "<cdf-cluster>"
  client_id = "<client-id>"

[powerops]
  read_dataset = "uc:000:powerops"
  write_dataset = "uc:000:powerops"
  monitor_dataset = "uc:po:monitoring"
```
and the `.secrets.toml`

```toml
[cognite]
  client_secret = "<client-secret>"

```

Values in `.secrets.toml` will overwrite those in `settings.toml`




In [14]:
# You can control which setting files are loaded through the environmental variable below.
# In this case, the setting files are located two levels above, in the root of the repository.
import os
from cognite.powerops._version import __version__
from cognite.powerops import PowerOpsClient


os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"

powerops = PowerOpsClient.from_settings()

WRITE_DATA_SET = powerops.datasets.write_dataset_id

print(__version__)  # Print the version of the package

The generated modules `cog_shop1`, `assets`, `afrr_bid`, and `day_ahead_bid` are deprecated and will be removed.
Module `shop` (SHOPRunAPI) is deprecated and will be removed.
Module `workflow` (DayaheadTriggerAPI) is deprecated and will be removed.
0.99.0



## Setting up a ShopCase instance that can be triggered:
1. Upload a (set of) file(s) for a shop case
2. Set up a ShopCase, connecting them with a scenario
3. Write the ShopCase instance. Verify that it was created using the SDK
4. Trigger a shop execution of that shop case


### Helpers for all types of case generation

In [15]:
from pathlib import Path
import datetime
from cognite.client import data_modeling as dm
from cognite.powerops.client._generated.v1.data_classes import (
    # These data classes are used to send data to CDF
    # There are non-write versions of these classes as well and they are used to read data from CDF
    ShopCaseWrite,  # This model contains everything needed to execute a SHOP run
    ShopFileWrite,  # A container that holds a reference to a file in CDF
    ShopModelWrite,  # A static model in a file reference + references to time series data
    ShopScenarioWrite,  # A way to modify run configurations for a given model
)
from cognite.powerops.client._generated.v1.data_classes._shop_case import ShopCase


# Directory where the example case files are located
EXAMPLE_CASES_ROOT = Path("example_case_files")


# helper method for all case generations
def upload_file(
    file_name: str,
    external_id: str | None = None,
    mime_type: str = "application/yaml",
) -> str:
    """For simplicity we we will use the file name as the external id,
    but it can be can use any unique string as the external id."""
    external_id = external_id or file_name
    file_path = (EXAMPLE_CASES_ROOT / file_name).resolve()
    file = powerops.cdf.files.upload(
        path=str(file_path),
        external_id=external_id,
        name=file_name,
        data_set_id=WRITE_DATA_SET,
        mime_type=mime_type,
        # Overwrite the file at the given external is if it already exists
        # This will also overwrite potentially existing metadata
        overwrite=True,
    )
    return file.external_id


def upload_shop_case(shop_case: ShopCaseWrite):
    # Add or update case. Do no merge a new case with an existing one
    powerops.v1.upsert(shop_case, replace=True, allow_version_increase=False)


def retrieve_shop_case(shop_case_external_id: str) -> ShopCase:
    # Retrieve a shop case based on external id
    # NB: Sometimes this can return None even if the case exists if it was just created
    return powerops.v1.day_ahead_bid.shop_case.retrieve(shop_case_external_id)


def trigger_shop_case(shop_case_external_id: str):
    # trigger
    powerops.cogshop.trigger_shop_case(shop_case_external_id)

### Example A: Complete case file

In this case, the `ShopScenario` as and its `ShopModel` are mostly superfluous. 
However, they are still added as nearly empty objects in order to set the SHOP version 


In [16]:
EXAMPLE_CASE_A = "example_case_a"


def create_shop_case_a() -> ShopCaseWrite:
    # NB! this files does not have any cut groups, shop result is reasonable
    file_reference = upload_file(file_name="a_case_with_commands.yaml")

    shop_case_write = ShopCaseWrite(
        externalId=EXAMPLE_CASE_A,  # unique identifier for the case, used to trigger SHOP execution
        # The time range SHOP is optimized over
        startTime=datetime.datetime(2023, 9, 14, 22),
        endTime=datetime.datetime(2023, 9, 24, 22),
        # The scenario is used to modify the run configuration of the model
        scenario=ShopScenarioWrite(
            name="dummy_scenario",
            # The model is a static model in a file reference + references to time series data
            model=ShopModelWrite(
                name="dummy_model",
                shop_version="15.6.1.0",
            ),
        ),
        # The files that are used in the case
        shopFiles=[
            ShopFileWrite(
                name="a_case_with_commands",
                label="",
                fileReference=file_reference,  # external id of the file on CDF
                isAscii=False,
                order=1,
            ),
        ],
    )
    return shop_case_write


case_a_write = create_shop_case_a()

In [92]:
upload_shop_case(case_a_write)

In [17]:
retrieve_shop_case(EXAMPLE_CASE_A)

In [99]:
trigger_shop_case(case_a_write.external_id)

### Example B: Almost complete case file

In this case, the `ShopScenario` as and its `ShopModel` are mostly superfluous. 
However, they are still added as nearly empty objects in order to set the SHOP version

A change here is that we extend the list in `ShopFile` and we need to specify the order that the files should be loaded into SHOP. 

Notice the label on the commands file. This is necessary as CogSHOP expects the commands to be labeled.  


In [6]:
EXAMPLE_CASE_B = "example_case_b"


def create_shop_case_b() -> ShopCaseWrite:
    # NB! this case does not have any cut groups, shop result is reasonable
    file_reference_case = upload_file(file_name="b_case_without_commands.yaml")
    file_reference_commands = upload_file(file_name="b_commands.yaml")

    shop_case_write = ShopCaseWrite(
        externalId=EXAMPLE_CASE_B,  # unique identifier for the case, used to trigger SHOP execution
        # The time range SHOP is optimized over
        startTime=datetime.datetime(2023, 9, 14, 22),
        endTime=datetime.datetime(2023, 9, 24, 22),
        # The scenario is used to modify the run configuration of the model
        scenario=ShopScenarioWrite(
            name="dummy_scenario",
            # The model is a static model in a file reference + references to time series data
            model=ShopModelWrite(
                name="dummy_model",
                shop_version="15.6.1.0",
            ),
        ),
        # The files that are used in the case
        shopFiles=[
            ShopFileWrite(
                name="b_case_without_commands",
                label="",
                fileReference=file_reference_case,  # external id of the file on CDF
                isAscii=False,
                order=1,
            ),
            ShopFileWrite(
                name="b_commands",
                label="commands",
                fileReference=file_reference_commands,  # external id of the file on CDF
                isAscii=False,
                order=2,
            ),
        ],
    )
    return shop_case_write

case_b_write = create_shop_case_b()

In [101]:
upload_shop_case(case_b_write)

In [7]:
retrieve_shop_case(EXAMPLE_CASE_B)

In [103]:
trigger_shop_case(case_b_write.external_id)

## Using the generated SDK to view a ShopResult instance

1. Querying for ShopResult instance, given their ShopCase external id
2. Using the external ID of a ShopResult to retrieve and inspect it  

In [10]:
from cognite.client import data_modeling as dm
from cognite.powerops.client._generated.v1.data_classes import ShopResultList


# Query the shop results based on the external id of the shop case
def result_instance_query(
    shop_case_external_id: str | list[str],
    external_id_prefix: str | None = None,
    limit: int | None = None,
    filter: dm.Filter | None = None,
) -> ShopResultList:
    """
    Query the shop results based on the external id of the shop case

    Parameters:
    shop_case_external_id: str | list[str]
        External id(s) of the shop cases that were triggered
    external_id_prefix: str | None
        Prefix of the external id(s)
    limit: int | None
        Maximum number of results to return. Defaults to 3
    filter: dm.Filter | None
        For advanced filtering in the case that the other parameters insufficient
        Refer to the Cognite Data Modeling documentation for more details

    Returns:
        ShopResultList

    """
    # Generate the query
    call_query = powerops.v1.day_ahead_bid.shop_result(
        case=shop_case_external_id,
        external_id_prefix=external_id_prefix,
        limit=limit,
        filter=filter,
    )
    # Execute and return the result
    return call_query.query()

In [11]:
# We convert the result to a pandas dataframe for easier data handling
all_results_df_1 = result_instance_query(
    shop_case_external_id=[EXAMPLE_CASE_A, EXAMPLE_CASE_B], limit=5
).to_pandas()

all_results_df_1

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,alerts,output_time_series,node_type,data_record


In [12]:
import pandas as pd
from cognite.powerops.client._generated.v1.data_classes import ShopResult, ShopResultList


# Retrieve a specific shop result instance based on the external id of the shop result
def result_instance_retrieve(external_id: str| list[str]) -> ShopResultList:
    """Returns a list of ShopResult instances based on the external id of the shop result"""
    instance = powerops.v1.shop_based_day_ahead_bid_process.shop_result.retrieve(
        external_id=external_id
    )
    return instance

In [13]:
# Get the row where the case is EXAMPLE_CASE_A
case_a_result: pd.DataFrame = all_results_df_1.loc[
    all_results_df_1["case"] == EXAMPLE_CASE_A
]


# Retrieve the first result instance from the dataframe above. This guarantees that the result exists
shop_result_a: ShopResult = result_instance_retrieve(case_a_result.external_id)[0]
shop_result_a

IndexError: list index out of range

In the case that a case was triggered multiple times, several `ShopResult`-instances will exist with a link with the same `case` link.

We can figure out which result was the most recently generated by looking at the `data_record` column. 

We will re-trigger another run of `EXAMPLE_CASE_B` to demonstrate

In [69]:
trigger_shop_case(EXAMPLE_CASE_B)

In [74]:
# We convert the result to a pandas dataframe for easier data handling
all_results_df_2 = result_instance_query(
    shop_case_external_id=[EXAMPLE_CASE_A, EXAMPLE_CASE_B], limit=5
).to_pandas()

all_results_df_2

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,alerts,output_time_series,node_type,data_record
0,power_ops_instances,shop_result__c828ca59-504f-4a63-be1a-e7f24a769e58,example_case_a,"{'total': -19927822739.279236, 'load_value': 0...",a_case_with_commands.yaml,POWEROPS_SHOP_post-run-c828ca59-504f-4a63-be1a...,POWEROPS_SHOP_shop-c828ca59-504f-4a63-be1a-e7f...,POWEROPS_SHOP_cplex-c828ca59-504f-4a63-be1a-e7...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-07..."
1,power_ops_instances,shop_result__e5f832cb-bc03-4419-835b-99ab80bc5471,example_case_b,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-e5f832cb-bc03-4419-835b-...,POWEROPS_SHOP_post-run-e5f832cb-bc03-4419-835b...,POWEROPS_SHOP_shop-e5f832cb-bc03-4419-835b-99a...,POWEROPS_SHOP_cplex-e5f832cb-bc03-4419-835b-99...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-07..."
2,power_ops_instances,shop_result__17dfb4b2-c746-43cf-a773-40118f9622ad,example_case_b,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-17dfb4b2-c746-43cf-a773-...,POWEROPS_SHOP_post-run-17dfb4b2-c746-43cf-a773...,POWEROPS_SHOP_shop-17dfb4b2-c746-43cf-a773-401...,POWEROPS_SHOP_cplex-17dfb4b2-c746-43cf-a773-40...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-07..."


In [87]:
# Get the row where the case is EXAMPLE_CASE_A
case_b_results: pd.DataFrame = all_results_df_2.loc[
    all_results_df_1["case"] == EXAMPLE_CASE_B
]
case_b_results

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,alerts,output_time_series,node_type,data_record
1,power_ops_instances,shop_result__e5f832cb-bc03-4419-835b-99ab80bc5471,example_case_b,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-e5f832cb-bc03-4419-835b-...,POWEROPS_SHOP_post-run-e5f832cb-bc03-4419-835b...,POWEROPS_SHOP_shop-e5f832cb-bc03-4419-835b-99a...,POWEROPS_SHOP_cplex-e5f832cb-bc03-4419-835b-99...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-07..."
2,power_ops_instances,shop_result__17dfb4b2-c746-43cf-a773-40118f9622ad,example_case_b,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-17dfb4b2-c746-43cf-a773-...,POWEROPS_SHOP_post-run-17dfb4b2-c746-43cf-a773...,POWEROPS_SHOP_shop-17dfb4b2-c746-43cf-a773-401...,POWEROPS_SHOP_cplex-17dfb4b2-c746-43cf-a773-40...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-07..."


In [81]:
shop_results_b: ShopResultList = result_instance_retrieve(case_b_results.external_id)
shop_results_b

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,alerts,output_time_series,node_type,data_record
0,power_ops_instances,shop_result__e5f832cb-bc03-4419-835b-99ab80bc5471,example_case_b,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-e5f832cb-bc03-4419-835b-...,POWEROPS_SHOP_post-run-e5f832cb-bc03-4419-835b...,POWEROPS_SHOP_shop-e5f832cb-bc03-4419-835b-99a...,POWEROPS_SHOP_cplex-e5f832cb-bc03-4419-835b-99...,[shop_penalty_report_dummy_scenario_2023-09-14...,None,None,"{'version': 1, 'last_updated_time': 2024-10-07..."
1,power_ops_instances,shop_result__17dfb4b2-c746-43cf-a773-40118f9622ad,example_case_b,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-17dfb4b2-c746-43cf-a773-...,POWEROPS_SHOP_post-run-17dfb4b2-c746-43cf-a773...,POWEROPS_SHOP_shop-17dfb4b2-c746-43cf-a773-401...,POWEROPS_SHOP_cplex-17dfb4b2-c746-43cf-a773-40...,[shop_penalty_report_dummy_scenario_2023-09-14...,None,None,"{'version': 1, 'last_updated_time': 2024-10-07..."


In [86]:
for result in shop_results_b:
    print(f"Result with external ID {result.external_id} was last updated at {result.data_record.last_updated_time.isoformat()}")


Result with external ID shop_result__e5f832cb-bc03-4419-835b-99ab80bc5471 was last updated at 2024-10-07T13:53:02.820000+00:00
Result with external ID shop_result__17dfb4b2-c746-43cf-a773-40118f9622ad was last updated at 2024-10-07T14:08:17.029000+00:00


## Cleaning up the instances that were created

Removing instances can be deleted by using the powerops client and the `external_id`s of everything we have created 

In [88]:
shop_case_external_ids = [EXAMPLE_CASE_A, EXAMPLE_CASE_B]
shop_result_external_ids = all_results_df_2.external_id.to_list()
all_external_ids = shop_case_external_ids + shop_result_external_ids

powerops.v1.delete(external_id=all_external_ids)

InstancesDeleteResult(nodes=[NodeId(space='power_ops_instances', external_id='example_case_a'), NodeId(space='power_ops_instances', external_id='example_case_b'), NodeId(space='power_ops_instances', external_id='shop_result__c828ca59-504f-4a63-be1a-e7f24a769e58'), NodeId(space='power_ops_instances', external_id='shop_result__e5f832cb-bc03-4419-835b-99ab80bc5471'), NodeId(space='power_ops_instances', external_id='shop_result__17dfb4b2-c746-43cf-a773-40118f9622ad')], edges=[])